In [ ]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt

from keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from keras.callbacks import ModelCheckpoint 
from keras.metrics import TopKCategoricalAccuracy

%matplotlib inline

### Define Parameters/Model

In [ ]:
# select model name; other option is 'MobileNetV3Large'
MODEL_NAME = 'InceptionResNetV2' 

# Set image width, height
if MODEL_NAME == 'InceptionResNetV2':
    IMG_WIDTH, IMG_HEIGHT = 299, 299
    from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
    
if MODEL_NAME == 'MobileNetV3Large':
    IMG_WIDTH, IMG_HEIGHT = 224, 224 
    from keras.applications.mobilenet_v3 import MobileNetV3Large, preprocess_input

# set model parameters
BATCH_SIZE=32

In [ ]:
# Supercategories:'Plant', 'Animalia', 'Arachnid', 'Mollusk', 'Mammal', 'Fungi', 'Reptile', 'Insect', 'Ray-finned Fishe', 'Birds', 'Amphibia'
# set train directory
SUPERCATEGORY = 'Birds'
IMG_TRAIN_DIR = os.path.join('./data/train_supercategory/', SUPERCATEGORY)
NUM_CLASSES = len(os.listdir(IMG_TRAIN_DIR))
print(f'There are {NUM_CLASSES} classes in {IMG_TRAIN_DIR}')

IMG_VAL_DIR = os.path.join('./data/val_supercategory/', SUPERCATEGORY)
IMG_TRAIN_MINI_DIR = os.path.join('./data/train_supercategory/', SUPERCATEGORY)

MODEL_SAVEDIR = os.path.join('./models/full/', SUPERCATEGORY)
print('Model Savedir:', MODEL_SAVEDIR)

### Build DataGenerators

In [ ]:
# build train datagen
train_datagen = ImageDataGenerator(
    preprocessing_function = preprocess_input, # preprocess input already scales to [-1,1]
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
)

# attach the generator
train_generator = train_datagen.flow_from_directory(
    directory = IMG_TRAIN_DIR,
    target_size = (IMG_WIDTH, IMG_HEIGHT),
    color_mode='rgb',
    batch_size = BATCH_SIZE,
    class_mode = 'categorical',
    shuffle=True
)
assert '.ipynb_checkpoints' not in train_generator.class_indices

In [ ]:
# same for val; no preprocessing for this one..
val_datagen = ImageDataGenerator(
    preprocessing_function = preprocess_input, # preprocess input already scales to [-1,1]
)

# attach the generator
val_generator = val_datagen.flow_from_directory(
    directory = IMG_VAL_DIR,
    target_size = (IMG_WIDTH, IMG_HEIGHT),
    color_mode='rgb',
    batch_size = BATCH_SIZE,
    class_mode = 'categorical',
    shuffle=True
)
assert '.ipynb_checkpoints' not in val_generator.class_indices

In [ ]:
# and now add on mini
train_mini_datagen = ImageDataGenerator(
    preprocessing_function = preprocess_input, # preprocess input already scales to [-1,1]
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
)

# attach the datagen; we will try to build a model with birds first
train_mini_generator = train_mini_datagen.flow_from_directory(
    directory = IMG_TRAIN_MINI_DIR,
    target_size = (IMG_WIDTH, IMG_HEIGHT),
    color_mode='rgb',
    batch_size = BATCH_SIZE,
    class_mode = 'categorical',
    subset='training',
    shuffle=True
)

assert '.ipynb_checkpoints' not in train_mini_generator.class_indices

### Build Model

In [ ]:
if MODEL_NAME == 'InceptionResNetV2':
    base_model=InceptionResNetV2(
        weights='imagenet',
        include_top=False, 
        input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)
    )

if MODEL_NAME == 'MobileNetV3Large':
    # import basemodel Inception ResnetV2 with imagenet weights
    base_model=MobileNetV3Large(
        weights='imagenet',
        include_top=False, 
        input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)
    )
    
print(f"Number of layers: {len(base_model.layers)}")
base_model.output.shape

In [ ]:
# attach on top layer
x = base_model.output
# flatten and dense
x = Flatten()(x)
output = Dense(NUM_CLASSES, activation='softmax')(x) # number of classes
model = Model(inputs=base_model.input, outputs=output)

## Train Model
Approach to training
-  fine-tune train top layer (full dataset)
-  fine-tune train all layers (full dataset)
-  fine-tune top layer (mini balanced dataset)
-  fine-tune all layers (mini balanced dataset)

### Train Top Layer with Full Dataset

In [ ]:
# freeze all base layers at first
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# decay every 2 epochs (2* each step; a step is where each gradient update happens)
lr_schedule = ExponentialDecay(initial_learning_rate=0.045, decay_steps=(2*train_generator.samples//BATCH_SIZE) , decay_rate=0.94)

# compile model
top_k_metric = TopKCategoricalAccuracy(k=5)

# compile model
model.compile(
    optimizer=RMSprop(momentum=0.9, learning_rate=lr_schedule), 
    loss='categorical_crossentropy', 
    metrics=['accuracy', top_k_metric]
)

In [ ]:
# save best checkpoints
print(f'Saving models in {MODEL_SAVEDIR}')
checkpoint = ModelCheckpoint(filepath=MODEL_SAVEDIR, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

# fit model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples//BATCH_SIZE,
    validation_data=val_generator,
    validation_steps=val_generator.samples//BATCH_SIZE,
    epochs = 20,
    callbacks=[checkpoint]
    )

In [ ]:
# return evaluation metrics
score = model.evaluate(val_generator)

print ("%s: %.2f%%" % (model.metrics_names[0], score[0]*100))
print ("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

In [ ]:
print(score)

In [ ]:
# plot accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# plot loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

### Train Full Model With Dataset

In [ ]:
# unfreeze and train on all layers
for layer in model.layers:
    layer.trainable = True
    
# re-compile model to refresh learning rate scheduler
model.compile(
    optimizer=RMSprop(momentum=0.9, learning_rate=lr_schedule), 
    loss='categorical_crossentropy', 
    metrics=['accuracy', top_k_metric]
)

In [ ]:
# Fine-tune
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples//BATCH_SIZE,
    validation_data=val_generator,
    validation_steps=val_generator.samples//BATCH_SIZE,
    epochs = 100,
    callbacks=callbacks_list
    )


In [ ]:
# return evaluation metrics
score = model.evaluate(val_generator)
print(score)

print ("%s: %.2f%%" % (model.metrics_names[0], score[0]*100))
print ("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

In [ ]:
# plot accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# plot loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()